In [ ]:
!pip install transformers

In [ ]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 13.3 MB/s eta 0:00:00


In [ ]:
# prompt: read hin_train.json and convert it to a dataframe by iterating over all rows

import pandas as pd
import json

def read_json_to_dataframe(file_path):
    data = []
    with open(file_path, 'r', encoding='utf-8') as file:
        for line in file:
            try:
                data.append(json.loads(line))
            except json.JSONDecodeError as e:
                print(f"Error decoding JSON line: {e}")
                # You might want to handle the error differently, e.g., skip the line
                continue
    return pd.DataFrame(data)

# Example usage (assuming 'hin_train.json' is in the current directory)
try:
    train_df = read_json_to_dataframe('hin_train.json')
    print(train_df.head())  # Print the first few rows of the DataFrame
except FileNotFoundError:
    print("Error: hin_train.json not found. Please ensure the file exists in the current directory.")



In [ ]:
# prompt: read hin_train.json and convert it to a dataframe by iterating over all rows

import pandas as pd
import json

def read_json_to_dataframe(file_path):
    data = []
    with open(file_path, 'r', encoding='utf-8') as file:
        for line in file:
            try:
                data.append(json.loads(line))
            except json.JSONDecodeError as e:
                print(f"Error decoding JSON line: {e}")
                # You might want to handle the error differently, e.g., skip the line
                continue
    return pd.DataFrame(data)

# Example usage (assuming 'hin_train.json' is in the current directory)
try:
    test_df = read_json_to_dataframe('hin_test.json')
    print(test_df.head())  # Print the first few rows of the DataFrame
except FileNotFoundError:
    print("Error: hin_train.json not found. Please ensure the file exists in the current directory.")



In [ ]:
train_df = train_df.rename(columns={'native word': 'devnagari', 'english word': 'romanized'})
print(train_df.head())  # Print the first few rows of the DataFrame

In [ ]:
test_df = test_df.rename(columns={'native word': 'devnagari', 'english word': 'romanized'})
print(test_df.head())  # Print the first few rows of the DataFrame

In [ ]:
from transformers import AutoTokenizer

# Load the mT5 tokenizer
tokenizer = AutoTokenizer.from_pretrained("google/mt5-base")

In [ ]:
def preprocess_function(examples):
    model_inputs = tokenizer(
        examples["romanized"],
        max_length=128,  # Set a maximum length for input sequences
        padding="max_length",  # Pad shorter sequences to the maximum length
        truncation=True,  # Truncate longer sequences to the maximum length
    )
    # If you have labels, apply the same preprocessing to them
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(
            examples["devnagari"],
            max_length=64,  # Set a maximum length for labels
            padding="max_length",
            truncation=True,
        )
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [ ]:
from datasets import Dataset

# Convert train and test DataFrames to Datasets
train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)

In [ ]:
print(train_dataset[0])

In [ ]:
tokenized_train = train_dataset.map(preprocess_function, batched=True)
tokenized_test = test_dataset.map(preprocess_function, batched=True)

In [ ]:
from transformers import Seq2SeqTrainingArguments

training_args = Seq2SeqTrainingArguments(
    output_dir="./mt5-finetuned",  # Directory to save the model
    evaluation_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=3,
    predict_with_generate=True,
    logging_dir="./logs",
    logging_steps=50,
)

In [ ]:
from transformers import AutoModelForSeq2SeqLM, Seq2SeqTrainer

# Load the mT5 model
model = AutoModelForSeq2SeqLM.from_pretrained("google/mt5-base")

# Initialize the Seq2SeqTrainer
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_test,
    tokenizer=tokenizer,
)

In [ ]:
trainer.train()

In [ ]:
model.save_pretrained("./mt5-transliteration")
tokenizer.save_pretrained("./mt5-transliteration")